# Main 
Alireza: I envisioned that this notebook can be the main training pipeline that calls on various function to generate clean features for the given list of item_id. These dataframes will be merged/joined on item_id and that final dataframe can be used for training and ensembling.

*Note: if this format is acceptable by you, it is preferrable to always pull the latest version of this notebook and then make a pull request (PR) for changing it as it is one of the main files where collaboration happens hence a high chance of unwanted merge conflicts!

Zexuan: Agree with Alizera above. This main2.ipynb will be used for final production

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sqlite3
from sqlalchemy.engine import create_engine

from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit

from tpot import TPOTClassifier

import tpot_functions 
import periods 


Define some key variables and numbers

In [2]:
target = 'deal_probability'
LIMIT = 10000
TIMEOUT_MINS = 30
SCORING = 'neg_mean_squared_error'
RS = 23

In [22]:
train = pd.read_csv('data/train.csv').dropna()
train_periods = pd.read_csv('data/periods_aggregate_features_items.csv').dropna()

In [24]:
train = pd.concat([train[['item_id', 'price', target]], 
                  pd.get_dummies(train[['category_name', 'user_type', target]])], 
                  axis=1) 

#train = pd.concat(pd.get_dummies(train[['category_name', 'user_type', target]]), train[['item_id', 'price', target]]
      


In [26]:
train

,item_id,price,deal_probability,deal_probability,category_name_Автомобили,category_name_Гаражи и машиноместа,category_name_Детская одежда и обувь,"category_name_Дома, дачи, коттеджи",category_name_Квартиры,category_name_Недвижимость за рубежом,"category_name_Одежда, обувь, аксессуары",user_type_Company,user_type_Private,user_type_Shop
4,7c90be56d2ab,40000.0,0.20797,0.20797,1,0,0,0,0,0,0,0,1,0
7,6b71309d6a8a,500.0,0.80323,0.80323,0,0,0,0,0,0,1,0,1,0
8,c5b969cb63a2,500.0,0.00000,0.00000,0,0,0,0,0,0,1,0,1,0
9,b1570962e68c,400.0,0.00000,0.00000,0,0,1,0,0,0,0,1,0,0
10,d5480bb4a6e4,18000.0,0.58853,0.58853,0,0,0,0,1,0,0,0,1,0
15,4b6abd0a5921,800.0,0.00000,0.00000,0,0,0,0,0,0,1,0,1,0
16,7896ef8fe482,140000.0,0.32100,0.32100,1,0,0,0,0,0,0,1,0,0
18,071e1ed13c5c,1000.0,0.00000,0.00000,0,0,0,0,0,0,1,0,1,0
20,ea12aec32ec3,1748000.0,0.00000,0.00000,0,0,0,0,1,0,0,1,0,0
22,de310e6aae86,249000.0,0.10334,0.10334,1,0,0,0,0,0,0,0,1,0


In [27]:
train_new = pd.merge(train, train_periods, on='item_id')
train_new

,item_id,price,deal_probability,deal_probability,category_name_Автомобили,category_name_Гаражи и машиноместа,category_name_Детская одежда и обувь,"category_name_Дома, дачи, коттеджи",category_name_Квартиры,category_name_Недвижимость за рубежом,...,median_date_to_isholiday,median_date_to_wday,median_date_to_yday,start_date_isholiday,start_date_wday,start_date_yday,end_date_isholiday,end_date_wday,end_date_yday,nitems


In [20]:
X = (train.drop(target, axis=1)).values[:LIMIT]
y = train[target].values[:LIMIT]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.75, test_size=0.25)

In [21]:
# TimeSeriesSplit NOT RIGHT YET
#tss = TimeSeriesSplit(n_splits=4)
#train_index, test_index = list(tss.split(X))[-1]
#X_train, X_test = X[train_index], X[test_index]
#y_train, y_test = y[train_index], y[test_index]

In [22]:
#tpot_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,
#                      random_state=42, verbosity=2)

In [23]:
tpot_optimizer = tpot_functions.TpotAutoml(mode='regression', max_time_mins=TIMEOUT_MINS, scoring=SCORING, random_state=RS, n_jobs=-1)


In [24]:
tpot_optimizer.fit(X_train, y_train)

In [25]:
top_scores = tpot_optimizer.get_top_models(return_scores=True)
print('\ntop cv scores:')
print(top_scores)
print('\ntop models')
print(tpot_optimizer.top_models)
print('\nthe best test score:')
test_score = tpot_optimizer.score(X_test, y_test)
print(test_score)


top cv scores:
OrderedDict([('LassoLarsCV', -0.03711585502407313), ('ExtraTreesRegressor', -0.03726474075425672), ('ElasticNetCV', -0.037357401715009425), ('XGBRegressor', -0.0374016365967508), ('DecisionTreeRegressor', -0.03765657702266903), ('RandomForestRegressor', -0.03765816650528318), ('GradientBoostingRegressor', -0.037703542021451944), ('RidgeCV', -0.038065861169076266), ('KNeighborsRegressor', -0.03864887021668067), ('LinearSVR', -0.04488545310200571)])

top models
OrderedDict([('LassoLarsCV', {'generation': 'INVALID', 'mutation_count': 53, 'crossover_count': 12, 'predecessor': ('LassoLarsCV(XGBRegressor(LinearSVR(MinMaxScaler(MaxAbsScaler(ElasticNetCV(MinMaxScaler(DecisionTreeRegressor(input_matrix, DecisionTreeRegressor__max_depth=3, DecisionTreeRegressor__min_samples_leaf=17, DecisionTreeRegressor__min_samples_split=6)), ElasticNetCV__l1_ratio=0.55, ElasticNetCV__tol=0.01))), LinearSVR__C=10.0, LinearSVR__dual=True, LinearSVR__epsilon=0.001, LinearSVR__loss=epsilon_insensi